In [30]:
import json
id_intput = dict()
id_output = dict()
HEAD = "I want you to act as a program developer.\nDo not write any explanations or other words, just reply with the complete function."
lang = 'python'

output_result_file = f'human_eval\\{lang}\\result_java.jsonl'
input_chat_gpt = f'human_eval\\{lang}\\queue_java_api_xai.jsonl'
output_chat_gpt = f'human_eval\\{lang}\\output_queue_java_api_xai.jsonl'

output_result_file = f'human_eval\\{lang}\\result_{lang}.jsonl'
input_chat_gpt = f'human_eval\\{lang}\\queue_api_xai_python_humaneval.jsonl'
output_chat_gpt = f'human_eval\\{lang}\\output_queue_api_xai_python_humaneval.jsonl'


def process_java(source):
    source = source.strip()
    if source.endswith('}'):
        source = source[:-1].strip()
    if source.endswith('}'):
        source = source[:-1].strip()
        
    return source

with open(input_chat_gpt) as f:
    data_input = [json.loads(l.strip()) for l in f.readlines()]
    for obj in data_input:
        id_intput[obj['custom_id']] = obj['body']['messages'][0]['content'].replace(HEAD,'').strip()
with open(output_chat_gpt) as f:
    data = [json.loads(l.strip()) for l in f.readlines()]
    for obj in data:
        source = obj['response']['body']['choices'][0]['message']['content']
        if lang == 'java':
            id_output[obj['custom_id']] = process_java(source)
        else:
            id_output[obj['custom_id']] = source
result = list()
for k,v in id_output.items():
    if lang == 'python':
        result.append({
            'task_id' : k,
            'generation' :  v,
            'prompt' : id_intput[k]
        })
    else:
        result.append({
            'task_id' : k,
            'generation' : id_intput[k] +'\n' + v,
            'prompt' : id_intput[k]
        })
with open(output_result_file,'w+') as f:
    for obj in result:
        f.writelines(json.dumps(obj)+'\n')


In [ ]:
for k,v in id_output.items():
    print(id_intput[k])
    print(v)